# Imports

In [ ]:
import pandas as pd
import numpy as np
import sklearn
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report, confusion_matrix  
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import RandomizedSearchCV

# Load Data

In [ ]:
data = pd.read_csv('data/DC_Properties.csv')

# Data Exploration

In [ ]:
data.shape # we have 49 columns and 158957 rows

In [ ]:
missing_values = data.isnull().sum()

print("Row Count: " + str(data.shape[0]))
print()
print(missing_values)

In [ ]:
data[data["STYLE"].notnull()].shape # 52.261 rows don't have a value for the "STYLE" column

In [ ]:
data[data["STRUCT"].notnull() | data["STYLE"].notnull() | data["GRADE"].notnull() | data["CNDTN"].notnull() | data["EXTWALL"].notnull() | data["ROOF"].notnull() | data["INTWALL"].notnull()].shape # the same rows don't have a value for: "STRUCT", "GRADE", "CNDTN", "EXTWALL", "ROOF", "INTWALL"

In [ ]:
# command copied form data cleaning
data_dropped_rows = data[data["STRUCT"].notnull() | data["STYLE"].notnull() | data["GRADE"].notnull() | data["CNDTN"].notnull() | data["EXTWALL"].notnull() | data["ROOF"].notnull() | data["INTWALL"].notnull()]

In [ ]:
missing_values = data_dropped_rows.isnull().sum()

print("Row Count: " + str(data_dropped_rows.shape[0]))
print()
print(missing_values)

1) no row has a value for the cmplx_num and the living_gba => useless
2) there are many missing values for the sale_date and the yr_rmdl => maybe they aren't missing, it could be the case, that these houses were never remodeled and/ or sold before, in the following we will probably handle them this way.

There is a column called "Unnamed: 0", lets see what's in there.

In [ ]:
data["Unnamed: 0"]

It's just the same as the index (probably the unique id column).

In [ ]:
data["STATE"].unique() 

In [ ]:
data["CITY"].unique()

Since our dataset is only about Washington DC these columns habe no use.

In [ ]:
data["ZIPCODE"].unique()

The ZIPCODE on the otherside contains some value

In [ ]:
data["FULLADDRESS"].unique()

In [ ]:
data["FULLADDRESS"].unique().shape

In [ ]:
data.shape[0] - data["FULLADDRESS"].isnull().sum()

The column "FULLADDRESS" has only unique text values, the computer can't process them. Either you split them up (street and house number or just leave them out because the streetnames aren't that useful either)

In [ ]:
data["NATIONALGRID"].unique()

In [ ]:
data["NATIONALGRID"].unique().shape

In [ ]:
data.shape[0] - data["NATIONALGRID"].isnull().sum()

The column "NATIONALGRID" has the same problem, as the one above.

In [ ]:
data["GIS_LAST_MOD_DTTM"].unique()

This row says when the row of the dataset was last modified, it's useless for us.

In [ ]:
data["SALEDATE"].unique()

This row contains the date, the building was sold. The way it is saved is not good to compute, we will convert it later.

In [ ]:
figure(num=None, figsize=(8, 6), dpi=80, facecolor='w', edgecolor='k')
plt.ticklabel_format(style = 'plain')

data.boxplot(column=["PRICE"])
plt.show()

In [ ]:
data.sort_values(by="PRICE")["PRICE"]

The price of some buildings is suspicous, we have some verry verry high prices (140.000.000$) and some realy low prices (1$) 

In [ ]:
figure(num=None, figsize=(8, 6), dpi=80, facecolor='w', edgecolor='k')
plt.ticklabel_format(style = 'plain')

data.loc[(data["PRICE"] <= 25000000) & (data["PRICE"] >= 60000)].boxplot(column=["PRICE"])
plt.show()

In [ ]:
data["AC"].unique()

A zero in there is strange value it stands for NULL, therefore we should replace it with NULL instead of 0.

In [ ]:
data["BLDG_NUM"].unique()

We haven't found anything about this column in the documentation and since it has only two possible values, it seems kind of useless.

# Data Cleaning

## remove rows with missing values

In [ ]:
# this command will throw away ~33% of the data (maybe we will keep the data and do two seperate tests, one where we will throw away the rows where the data is missing and one where we will throw away these columns)
# data = data[data["STRUCT"].notnull() | data["STYLE"].notnull() | data["GRADE"].notnull() | data["CNDTN"].notnull() | data["EXTWALL"].notnull() | data["ROOF"].notnull() | data["INTWALL"].notnull()]

# since our many concern is the price of the building, rows without the price have only a small to none value
data = data[data["PRICE"].notnull()]

# remove rows with strange prices
data = data.loc[(data["PRICE"] <= 25000000) & (data["PRICE"] >= 60000)]

# in these very few rows (~200) there are values missing
data = data[data["X"].notnull() & data["Y"].notnull() & data["QUADRANT"].notnull() & data["AYB"].notnull() & data["WARD"].notnull() & data["ASSESSMENT_NBHD"].notnull() & data["CENSUS_TRACT"].notnull() & data["LONGITUDE"].notnull() & data["LATITUDE"].notnull() & data["ZIPCODE"].notnull()]
data = data[data.ROOMS != 0]
data = data[data.AC != "0"]

## drop columns

In [ ]:
# this column has only one value "2018-07-22 18:01:43" => useless
data = data.drop(["GIS_LAST_MOD_DTTM"], axis=1)
# since the unique id is equal to the row number we don't need it
data = data.drop(["Unnamed: 0"], axis=1)
# many many missing values
data = data.drop(["LIVING_GBA", "CMPLX_NUM"], axis=1)
# these columns have nothing to say
data = data.drop(["STATE", "CITY", "SOURCE", "BLDG_NUM"], axis=1)
# these columns contain only unique texts which can't be computed
data = data.drop(["NATIONALGRID", "FULLADDRESS"], axis=1)
# in our other notebook we proved that there is a realy high (< 0.999) correlation between x, y and longitude and latititude because in both show the longitude and latitdude of a building
data = data.drop(["X", "Y"], axis=1)

## data insertion

In [ ]:
# set the missing sale date to the year it was build
data["SALEDATE"] = np.where(data["SALEDATE"].isnull(), data["AYB"], data["SALEDATE"])

# add a column wich says wether a building was remodeled and insert missing values in to the YR_RMDL column
data["WAS_REMODELED"] = np.where(data["YR_RMDL"].isnull(), 0, 1)
data["YR_RMDL"] = np.where(data["YR_RMDL"].isnull(), -1, data["YR_RMDL"])

## data convertion

In [ ]:
#convert saledate to datetime
data['SALEDATE'] = pd.to_datetime(data['SALEDATE'])
#Calculating the difference in years between Last Sale Date and Year Built
data['SalevYB']=data['SALEDATE'].dt.year - data['AYB']
#Calculating the difference in years between Last Sale Date and Year Improved
data['SalevYI']=data['SALEDATE'].dt.year - data['EYB']

data = data.drop(["SALEDATE", "EYB", "AYB"], axis=1)

## final results

In [ ]:
missing_values = data.isnull().sum()
print("All Data:")
print("Row Count: " + str(data.shape[0]))
print("Col Count: " + str(data.shape[1]))
print()
print(missing_values)

data_col = data.drop(["NUM_UNITS", "STORIES", "GBA", "STYLE", "STRUCT", "GRADE", "CNDTN", "EXTWALL", "ROOF", "INTWALL", "KITCHENS", "ASSESSMENT_SUBNBHD", "CENSUS_BLOCK"], axis=1)
missing_values_col = data_col.isnull().sum()
print()
print("--------------------------------------------------")
print()
print("Data with dropped columns:")
print("Row Count: " + str(data_col.shape[0]))
print("Col Count: " + str(data_col.shape[1]))
print()
print(missing_values_col)

data_row = data[data["STRUCT"].notnull() & data["STYLE"].notnull() & data["GRADE"].notnull() & data["CNDTN"].notnull() & data["EXTWALL"].notnull() & data["ROOF"].notnull() & data["INTWALL"].notnull()  & data["STORIES"].notnull() & data["KITCHENS"].notnull() & data["ASSESSMENT_SUBNBHD"].notnull()  & data["CENSUS_BLOCK"].notnull()]
missing_values_row = data_row.isnull().sum()
print()
print("--------------------------------------------------")
print()
print("Data with dropped rows:")
print("Row Count: " + str(data_row.shape[0]))
print("Col Count: " + str(data_row.shape[1]))
print()
print(missing_values_row)

## save to csv

In [ ]:
data_row.to_csv("data/data_cleaned_row.csv", index=False)
data_col.to_csv("data/data_cleaned_col.csv", index=False)

# Data Preparation

## subsample (for faster development -> remove (comment out) later)

## One Hot Encoding (needed for the nn)

In [ ]:
data_row_one_hot = pd.get_dummies(data_row, prefix = ["HEAT", "AC", "QUALIFIED", "STYLE", "STRUCT", "GRADE", "CNDTN", "EXTWALL", "INTWALL", "ROOF", "ZIPCODE", "ASSESSMENT_NBHD", "ASSESSMENT_SUBNBHD", "CENSUS_TRACT", "CENSUS_BLOCK", "WARD", "QUADRANT", "WAS_REMODELED"], columns = ["HEAT", "AC", "QUALIFIED", "STYLE", "STRUCT", "GRADE", "CNDTN", "EXTWALL", "INTWALL", "ROOF", "ZIPCODE", "ASSESSMENT_NBHD", "ASSESSMENT_SUBNBHD", "CENSUS_TRACT", "CENSUS_BLOCK", "WARD", "QUADRANT", "WAS_REMODELED"])
data_col_one_hot = pd.get_dummies(data_col, prefix = ["HEAT", "AC", "QUALIFIED", "ZIPCODE", "ASSESSMENT_NBHD", "CENSUS_TRACT", "WARD", "QUADRANT", "WAS_REMODELED"], columns = ["HEAT", "AC", "QUALIFIED", "ZIPCODE", "ASSESSMENT_NBHD", "CENSUS_TRACT", "WARD", "QUADRANT", "WAS_REMODELED"])

## remove categorical data (needed for regression analysis)

In [ ]:
data_row_no_cat = data_row
data_col_no_cat = data_col

In [ ]:
data_row_no_cat = data_row_no_cat.drop(["HEAT", "AC", "QUALIFIED", "STYLE", "STRUCT", "GRADE", "CNDTN", "EXTWALL", "INTWALL", "ROOF", "ZIPCODE", "ASSESSMENT_NBHD", "ASSESSMENT_SUBNBHD", "CENSUS_TRACT", "CENSUS_BLOCK", "WARD", "QUADRANT", "WAS_REMODELED"], axis=1)
data_col_no_cat = data_col_no_cat.drop(["HEAT", "AC", "QUALIFIED", "ZIPCODE", "ASSESSMENT_NBHD", "CENSUS_TRACT", "WARD", "QUADRANT", "WAS_REMODELED"], axis=1)

## split the data (x and y)

In [ ]:
x_row = data_row.drop(["PRICE"], axis=1)
x_row_one_hot = data_row_one_hot.drop(["PRICE"], axis=1)
x_row_no_cat = data_row_no_cat.drop(["PRICE"], axis=1)
y_row = data_row["PRICE"]

In [ ]:
x_col = data_col.drop(["PRICE"], axis=1)
x_col_one_hot = data_col_one_hot.drop(["PRICE"], axis=1)
x_col_no_cat = data_col_no_cat.drop(["PRICE"], axis=1)
y_col = data_col["PRICE"]

## split the data (train and test set)

In [ ]:
split_size_row = round(data_row.shape[0] * 0.7)
split_size_col = round(data_col.shape[0] * 0.7)

In [ ]:
x_row_train, x_row_test = np.split(x_row, [split_size_row], axis = 0)
x_row_one_hot_train, x_row_one_hot_test = np.split(x_row_one_hot, [split_size_row], axis = 0)
x_row_no_cat_train, x_row_no_cat_test = np.split(x_row_no_cat, [split_size_row], axis = 0)
y_row_train, y_row_test = np.split(y_row, [split_size_row], axis = 0)

In [ ]:
x_col_train, x_col_test = np.split(x_col, [split_size_col], axis = 0)
x_col_one_hot_train, x_col_one_hot_test = np.split(x_col_one_hot, [split_size_col], axis = 0)
x_col_no_cat_train, x_col_no_cat_test = np.split(x_col_no_cat, [split_size_col], axis = 0)
y_col_train, y_col_test = np.split(y_col, [split_size_col], axis = 0)

# Use Linear Regression

## train the model

In [ ]:
print("Start trianing")
linreg_row = LinearRegression(n_jobs = -1, normalize = True)
linreg_row.fit(x_row_no_cat_train, y_row_train)
print("End training")

In [ ]:
print("Start trianing")
linreg_col = LinearRegression(n_jobs = -1, normalize = True)
linreg_col.fit(x_col_no_cat_train, y_col_train)
print("End training")

## predict the test data

In [ ]:
y_row_lin_pred = linreg_row.predict(x_row_no_cat_test)

In [ ]:
y_col_lin_pred = linreg_col.predict(x_col_no_cat_test)

## analyze results

In [ ]:
row_lin_score = linreg_row.score(x_row_no_cat_test, y_row_test)
row_lin_msqe = mean_squared_error(y_row_test, y_row_lin_pred)
print("R2 score for the row dataset: " + str(row_lin_score))
print("Mean squared error for the row dataset: " + str(row_lin_msqe))

In [ ]:
figure(num=None, figsize=(8, 6), dpi=80, facecolor='w', edgecolor='k')
plt.ticklabel_format(style = 'plain')

plt.scatter(y_row_test, y_row_lin_pred)
plt.xlabel("Prices: $Y_i$")
plt.ylabel("Predicted prices: $\hat{Y}_i$")
plt.title("Prices vs Predicted prices: $Y_i$ vs $\hat{Y}_i$")
plt.show()

In [ ]:
col_lin_score = linreg_col.score(x_col_no_cat_test, y_col_test)
col_lin_msqe = mean_squared_error(y_col_test, y_col_lin_pred)
print("R2 score for the column dataset: " + str(col_lin_score))
print("Mean squared error for the row dataset: " + str(col_lin_msqe))

In [ ]:
figure(num=None, figsize=(8, 6), dpi=80, facecolor='w', edgecolor='k')
plt.ticklabel_format(style = 'plain')

plt.scatter(y_col_test, y_col_lin_pred)
plt.xlabel("Prices: $Y_i$")
plt.ylabel("Predicted prices: $\hat{Y}_i$")
plt.title("Prices vs Predicted prices: $Y_i$ vs $\hat{Y}_i$")
plt.show()

# Use Logistic Regression (dataset is to complex for my computer to compute this)

## train the model

In [ ]:
print("Start training")
logreg_row = LogisticRegression(n_jobs = -1, solver = "sag", max_iter=100000)
logreg_row.fit(x_row_no_cat_train, y_row_train)
print("Training finished")

In [ ]:
print("Start training")
logreg_col = LogisticRegression(n_jobs = -1, solver = "sag", max_iter=100000, normalize = True)
logreg_col.fit(x_col_train, y_col_train)
print("Training finished")

## predict the test data

In [ ]:
y_row_log_pred = logreg_row.predict(x_row_no_cat_test)

In [ ]:
y_col_log_pred = logreg_col.predict(x_col_no_cat_test)

## analyze results

In [ ]:
row_log_score = logreg_row.score(x_row_no_cat_test, y_row_test)
print("R2 score for the row dataset: " + str(row_log_score))

In [ ]:
plt.scatter(y_row_test, y_row_log_pred)
plt.xlabel("Prices: $Y_i$")
plt.ylabel("Predicted prices: $\hat{Y}_i$")
plt.title("Prices vs Predicted prices: $Y_i$ vs $\hat{Y}_i$")
plt.show()

In [ ]:
col_log_score = logreg_col.score(x_col_no_cat_test, y_col_test)
print("R2 score for the column dataset: " + str(col_log_score))

In [ ]:
plt.scatter(y_col_test, y_col_lin_pred)
plt.xlabel("Prices: $Y_i$")
plt.ylabel("Predicted prices: $\hat{Y}_i$")
plt.title("Prices vs Predicted prices: $Y_i$ vs $\hat{Y}_i$")
plt.show()

# KNN

In [ ]:
# # train the model
# print("Start training")
# knn_row = KNeighborsClassifier(n_neighbors = 2, n_jobs = -1)
# knn_row.fit(x_row_one_hot_train, y_row_train)
# print("Finished training")

# # predict the test data
# y_row_knn_pred = knn_row.predict(x_row_one_hot_test)

# # output
# print(classification_report(y_row_test, y_row_knn_pred))

In [ ]:
# # train the model
# print("Start training")
# knn_col = KNeighborsClassifier(n_neighbors = 2, n_jobs = -1)
# knn_col.fit(x_col_one_hot_train, y_col_train)
# print("Finished training")

# # predict the test data
# y_col_knn_pred = knn_col.predict(x_col_one_hot_test)

# # output
# print(classification_report(y_col_test, y_col_knn_pred))

# Random Forest

In [ ]:
param_dist = {'n_estimators': [int(x) for x in np.linspace(start = 100, stop = 500, num = 10)],
               'max_features': ['auto', 'sqrt'],
               'max_depth': [1, 20]}

print("Start training")
RFR = RandomForestRegressor(n_jobs = -1)
RFR_cv = RandomizedSearchCV(RFR, param_dist)
RFR_cv.fit(x_row_one_hot_train, y_row_train)
print("Finished training")
evaluate(RFR_cv.best_estimator_, x_row_one_hot_test, y_row_test)

In [ ]:
y_row_rf_predict = RFR.predict(x_row_one_hot_test)

In [ ]:
figure(num=None, figsize=(8, 6), dpi=80, facecolor='w', edgecolor='k')
plt.ticklabel_format(style = 'plain')

plt.scatter(y_row_test, y_row_rf_predict)
plt.xlabel("Prices: $Y_i$")
plt.ylabel("Predicted prices: $\hat{Y}_i$")
plt.title("Prices vs Predicted prices: $Y_i$ vs $\hat{Y}_i$")
plt.show()